In [1]:
from nba_seer import *

modules imported
functions defined


In [2]:
import pandas as pd
import datetime
import random

from sqlalchemy import create_engine

import sys
sys.path.append('/Users/wonderui/OneDrive/6_Module_Package')
sys.path.append('/Users/WangRui/OneDrive/6_Module_Package')
import hoop_pwd
pwd = hoop_pwd.password

In [3]:
# create sql conn and load game stats logs ----------------------

conn = create_engine('mysql+pymysql://root:%s@118.190.202.87:3306/nba_stats' % pwd)
game_stats_logs = pd.read_sql_table('game_stats_logs', conn)
game_stats_logs = game_stats_logs[game_stats_logs['GAME_TYPE'] != 'all_star']

print(str(len(game_stats_logs)) + ' player stats loaded.')

182297 player stats loaded.
players list loaded


In [4]:
# load player list ----------------------

all_players = nba_py.player.PlayerList(season='2017-18').info()

print('players list loaded')

players list loaded


In [25]:
def get_score(row):
    player_id = row['PERSON_ID']
    game_id = row['GAME_ID']
    stats = game_stats_logs[(game_stats_logs['PLAYER_ID'] == player_id) & 
                            (game_stats_logs['GAME_ID'] == game_id)].copy()
    #print(stats)
    stats['SCO'] = stats['PTS'] * 1 + stats['AST'] * 1.5 + \
    stats['OREB'] * 1 + stats['DREB'] * 0.7 + \
    stats['STL'] * 2 + stats['BLK'] * 1.8 + stats['TO'] * -1 + \
    stats['FGM'] * 0.4 + (stats['FGA'] - stats['FGM']) * -1 + stats['FG3M'] * 0.5
    return stats['SCO'].max()

In [26]:
game_date = game_stats_logs['GAME_DATE_EST'].drop_duplicates().apply(lambda x: x[:10])[-150:]

In [27]:
players = pd.DataFrame()
for i in random.sample(list(game_date), 10):
    j = datetime.datetime.strptime(i, "%Y-%m-%d").date()
    games = get_games(j)
    players = players.append(get_players_p(games, game_stats_logs))
    print(str(j) + ' complete!')
else:
    print(len(players), ' players total!')

2017-05-21 complete!
2017-01-18 complete!
2017-04-01 complete!
2016-12-26 complete!
2017-04-04 complete!
2017-05-16 complete!
2017-03-25 complete!
2017-02-09 complete!
2017-05-17 complete!
2017-04-08 complete!
1465  players total!


In [28]:
players['MA_20'] = players.apply(lambda x: get_ma(game_stats_logs, x, 20), axis = 1)
print('ma20 complete!')
players['MA_10'] = players.apply(lambda x: get_ma(game_stats_logs, x, 10), axis = 1)
print('ma10 complete!')
players['MA_5'] = players.apply(lambda x: get_ma(game_stats_logs, x, 5), axis = 1)
print('ma5 complete!')
players['MIN_20'] = players.apply(lambda x: get_min(game_stats_logs, x, 20), axis = 1)
print('min20 complete!')
players['MIN_10'] = players.apply(lambda x: get_min(game_stats_logs, x, 10), axis = 1)
print('min10 complete!')
players['MIN_5'] = players.apply(lambda x: get_min(game_stats_logs, x, 5), axis = 1)
print('min5 complete!')
players['MIN_COV_20'] = players.apply(lambda x: get_min_cov(game_stats_logs, x, 20), axis = 1)
print('min_cov_20 complete!')
players['SCO_COV_20'] = players.apply(lambda x: get_sco_cov(game_stats_logs, x, 20), axis = 1)
print('sco_cov_20 complete!')

players['EXP_SCO'] = round(players[['MA_20', 'MA_10', 'MA_5']].mean(axis = 1) * 
                           players[['MIN_20', 'MIN_10', 'MIN_5']].mean(axis = 1) / 36, 2)

players['ACT_SCO'] = players.apply(get_score, axis = 1)
players['SCO_DIFF'] = players['EXP_SCO'] - players['ACT_SCO']
players

ma20 complete!
ma10 complete!
ma5 complete!
min20 complete!
min10 complete!
min5 complete!
min_cov_20 complete!
sco_cov_20 complete!


,PERSON_ID,TEAM_ID,Location,GAME_ID,Against_Team_ID,MA_20,MA_10,MA_5,MIN_20,MIN_10,MIN_5,MIN_COV_20,SCO_COV_20,EXP_SCO,ACT_SCO,SCO_DIFF
0,2544,1610612739,HOME,0041600303,1610612738,43.60,45.59,47.70,40.47,41.42,39.64,0.139,0.204,51.35,14.9,3.645000e+01
1,201567,1610612739,HOME,0041600303,1610612738,27.33,26.13,28.56,31.55,31.07,29.35,0.180,0.420,23.28,40.1,-1.682000e+01
2,202684,1610612739,HOME,0041600303,1610612738,21.33,21.18,21.51,30.41,32.11,31.57,0.182,0.359,18.59,30.7,-1.211000e+01
3,2747,1610612739,HOME,0041600303,1610612738,14.79,13.39,16.17,29.29,25.74,23.97,0.238,0.554,10.81,18.5,-7.690000e+00
4,202681,1610612739,HOME,0041600303,1610612738,29.08,28.49,29.86,34.44,33.86,33.30,0.160,0.453,27.42,40.9,-1.348000e+01
5,202697,1610612739,HOME,0041600303,1610612738,13.40,18.25,23.01,21.22,18.90,20.47,0.370,1.029,10.22,10.6,-3.800000e-01
6,2594,1610612739,HOME,0041600303,1610612738,18.95,17.86,21.53,19.35,18.26,19.19,0.300,0.587,10.23,4.6,5.630000e+00
7,2210,1610612739,HOME,0041600303,1610612738,12.05,10.56,11.43,14.92,11.71,12.54,0.418,0.952,4.12,-2.0,6.120000e+00
8,101114,1610612739,HOME,0041600303,1610612738,20.29,25.31,20.60,17.88,15.69,15.46,0.424,0.764,10.02,1.7,8.320000e+00
9,101112,1610612739,HOME,0041600303,1610612738,28.46,29.63,30.94,16.65,13.09,11.41,0.436,0.332,11.31,NaN,NaN


In [29]:
players['SCO_DIFF'].describe()

count    1226.000000
mean       -0.618426
std         8.720016
min       -54.040000
25%        -5.912500
50%         0.245000
75%         4.960000
max        36.450000
Name: SCO_DIFF, dtype: float64